# Pre-process the data before running the customization job

>We recommend to run this notebook using 
>- kernel: `Data Science 3.0` or `Python 3`
>- instance size `ml.t3.medium` or greater

## Prepare the dataset

The dataset is based on a PDF file containing 1,625 European CapMarkets and Bank Finance terms.
Let's first download the dataset from [Latham & Watkins' website](https://www.lw.com/en/book-of-jargon/boj-european-capital-markets-and-bank-finance).

In [ ]:
import urllib.request

url = "https://www.lw.com/admin/Upload/Documents/Books%20of%20Jargon/Book-of-Jargon-European-Capital-Markets-and-Bank-Finance-2nd-Edition.pdf"
filename = "capmarkets-jargon"
filename_pdf = "capmarkets-jargon.pdf"

urllib.request.urlretrieve(url, filename_pdf)

('capmarkets-jargon.pdf', <http.client.HTTPMessage at 0x103d34c10>)

Since our dataset is in PDF format, we need to install PyPDF2 library to be able to extract text from PDF.

In [ ]:
!pip install PyPDF2

Now, let's define a function that converts each line of a PDF page into JSONL.

In [ ]:
terms_counter = 0
parts = []

### Construct JSONL from one PDF page

def visitor_body(text, cm, tm, fontDict, fontSize):
    global terms_counter
    
    if len(text) > 1:
        if(fontDict['/BaseFont'] == '/IYOEDH+CandidaStd-Bold'):
            # close previous term, if there's one
            if(parts):
                parts.append("\"}")
                parts.append("\n")


            terms_counter = terms_counter + 1
            
            parts.append("{\"input\":" + " \"" + text )
        else:
            parts.append(text)


Next, we iterate through each PDF page containing definitions converting them to JSONL and then save the output as capmarkets-jargon.jsonl file.

In [ ]:
from PyPDF2 import PdfReader

pdfReader = PdfReader(filename_pdf) 
start_page = 3
end_page = 186
terms_counter = 0

# clear all old items, if any
parts.clear()
output = ""
for i in range(start_page, end_page):
    page = pdfReader.pages[i]
    page.extract_text(visitor_text=visitor_body)

# close the last term
parts.append( "\"" + "}" )

print("Terms processed:", terms_counter)

output = "".join(parts)
with open(filename + ".jsonl", "w") as outfile:
    outfile.write(output)


Terms processed: 1624


Finally, let's upload the generated JSONL file to S3 so that it's accessible by Bedrock.

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()

In [ ]:
# Specify the path to the training data
train_data_path = filename + ".jsonl"

# Upload the training data to the specified S3 key prefix 'PreProcessed'
s3_train_data = sagemaker_session.upload_data(path=train_data_path, key_prefix='PreProcessed')

# Print a message indicating the successful upload
print(f"Uploaded {train_data_path} to {s3_train_data}")

Uploaded capmarkets-jargon.jsonl to s3://sagemaker-us-east-1-922650977840/PreProcessed/capmarkets-jargon.jsonl
